In [118]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.startswith('bike'):
            print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/av-ms-ds/bike-sharing-Readme.txt
/kaggle/input/av-ms-ds/bike-sharing-day.csv


In [119]:
import warnings
warnings.filterwarnings('ignore')

In [120]:

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import numpy as np
import pandas as pd

In [121]:
bike_df = pd.DataFrame(pd.read_csv("/kaggle/input/av-ms-ds/bike-sharing-day.csv"))
bike_df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [122]:
bike_df.shape

(730, 16)

In [123]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    int64  
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.4+ KB


In [124]:
bike_df.describe()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000
mean,365.500000,2.498630,0.500000,6.526027,0.028767,2.997260,0.683562,1.394521,20.319259,23.726322,62.765175,12.763620,849.249315,3658.757534,4508.006849
std,210.877136,1.110184,0.500343,3.450215,0.167266,2.006161,0.465405,0.544807,7.506729,8.150308,14.237589,5.195841,686.479875,1559.758728,1936.011647
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.424346,3.953480,0.000000,1.500244,2.000000,20.000000,22.000000
25%,183.250000,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,13.811885,16.889713,52.000000,9.041650,316.250000,2502.250000,3169.750000
50%,365.500000,3.000000,0.500000,7.000000,0.000000,3.000000,1.000000,1.000000,20.465826,24.368225,62.625000,12.125325,717.000000,3664.500000,4548.500000
75%,547.750000,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,26.880615,30.445775,72.989575,15.625589,1096.500000,4783.250000,5966.000000
max,730.000000,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,35.328347,42.044800,97.250000,34.000021,3410.000000,6946.000000,8714.000000


# Highlevel observations from the dataframe

* Dataset has 16 columns and 730 rows.

* Except one column (dteday: object), all other are either float(float64) or integer(int64) type.

* dteday column is object type, but it has to be of date data type.

* At the highlevel, there seems to be some fields that are categorical in nature, but in integer/float type.
 
* We will analyse and finalize whether to convert them to categorical(discrete) or treat as integer(continuous)

## Inspect Null or Missing values

In [125]:
# percentage of missing values in each column
round(100*(bike_df.isnull().sum()/len(bike_df)), 2).sort_values(ascending=False)

instant       0.0
dteday        0.0
season        0.0
yr            0.0
mnth          0.0
holiday       0.0
weekday       0.0
workingday    0.0
weathersit    0.0
temp          0.0
atemp         0.0
hum           0.0
windspeed     0.0
casual        0.0
registered    0.0
cnt           0.0
dtype: float64

In [126]:
round((bike_df.isnull().sum(axis=1)/len(bike_df))*100,2).sort_values(ascending=False)

729    0.0
0      0.0
1      0.0
2      0.0
3      0.0
      ... 
29     0.0
30     0.0
31     0.0
32     0.0
33     0.0
Length: 730, dtype: float64

There are no missing / Null values either in columns or rows

### Checking for duplicates and dropping the entire duplicate row if any

In [127]:
bike_df_copy = bike_df.copy()


bike_df_copy.drop_duplicates(subset=None, inplace=True)

In [128]:
bike_df_copy.shape

(730, 16)

In [129]:
bike_df.shape

(730, 16)

There is no reduction in rows in copied df since there is no duplicates found.

So we can conclude that there were zero duplicate values in the bike dataset.

Data Cleaning

Let us check for `value_counts()` for entire dataframe to identify any Unknown/Junk values present in the dataset.

In [130]:
bike_temp=bike_df.iloc[:,1:16]

In [131]:
bike_temp.head()

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [132]:
# util function to extract value counts for all the columns in a df
def extract_value_counts(df):
    for col in df:
        print("column name: ", col)
        print('+'*50,'\n\n\n',df[col].value_counts(ascending=False), '\n\n\n')

In [133]:
extract_value_counts(bike_df)

column name:  instant
++++++++++++++++++++++++++++++++++++++++++++++++++ 


 instant
730    1
1      1
2      1
3      1
4      1
      ..
30     1
31     1
32     1
33     1
34     1
Name: count, Length: 730, dtype: int64 



column name:  dteday
++++++++++++++++++++++++++++++++++++++++++++++++++ 


 dteday
31-12-2019    1
01-01-2018    1
02-01-2018    1
03-01-2018    1
04-01-2018    1
             ..
30-01-2018    1
31-01-2018    1
01-02-2018    1
02-02-2018    1
03-02-2018    1
Name: count, Length: 730, dtype: int64 



column name:  season
++++++++++++++++++++++++++++++++++++++++++++++++++ 


 season
3    188
2    184
1    180
4    178
Name: count, dtype: int64 



column name:  yr
++++++++++++++++++++++++++++++++++++++++++++++++++ 


 yr
0    365
1    365
Name: count, dtype: int64 



column name:  mnth
++++++++++++++++++++++++++++++++++++++++++++++++++ 


 mnth
1     62
3     62
7     62
5     62
12    62
10    62
8     62
4     60
9     60
6     60
11    60
2     56
Name: count,

After observing the above data we could conclude there is no null or unknown values in the dataframe.


### Removing redundant & unwanted columns

Based on a high-level review of the data and the data dictionary, the following variables can be removed from further analysis:

| Variable | Reason for Removal |
|---|---|
| **`instant`** | 📅 **Index Value**: It's merely an index and does not provide useful information for analysis. |
| **`dteday`** | 📆 **Date Column**: We already have separate columns for 'year' and 'month', so this column is redundant. |
| **`casual`** & **`registered`** | 🚴‍♂️ **Customer Counts**: These columns contain the count of bikes booked by different customer categories. Our objective is to find the total count of bikes, not by specific category. Additionally, we have created a new variable to represent the ratio of these customer types. |

We will save the new dataframe as `bike_refined_df` to preserve the original dataset for any future analysis or validation.



In [134]:
 #Lets delete all the columns which value is unique in nature by verifiying as shown below.

unique_columns = [col for col in bike_df.columns if bike_df[col].nunique() == len(bike_df)]
unique_columns

['instant', 'dteday']

In [135]:
bike_df.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

## Dropping casual and Registed as  they sumed to Cnt.

In [136]:
bike_df.drop(['casual','registered'], axis=1, inplace=True)

In [ ]:
bike_df.shape

In [ ]:
bike_refined_df = bike_df[['season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'cnt']]

In [ ]:
bike_refined_df.info()

## Perform the EDA on (bike_refined_df) Dataset.

In [ ]:
# function to create barplot related to categorical columns

def plot_bar_graphs(df,column):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.barplot(x=column, y='cnt', data=df)
    plt.title(f'Bar Plot of {column} vs cnt')
    
    plt.subplot(1, 2, 2)
    sns.barplot(x=column, y='cnt', data=df, hue='yr', palette='Set1')
    plt.title(f'Bar Plot of {column} vs cnt (2018 vs 2019)')
    plt.legend(title='yr', labels=['2018', '2019'])
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Lets check if any outliers present is numeric featires.
# raw box plots for indepent variables with continuous values
cols = ['temp', 'atemp', 'hum', 'windspeed']
plt.figure(figsize=(18,4))

k = 1
for col in cols:
    plt.subplot(1,4,k)
    sns.boxplot(y=col, data=bike_refined_df)
    k+=1

In [ ]:
# Line graph will that show the count vs month for year 2018 and 2019.
sns.lineplot(x = "mnth", y = "cnt", data=bike_refined_df,hue = "yr")
plt.show() 

In [ ]:
# Inferences:
# 1. From Line Graph we can see that target Variable cnt has increased from 2018 to 2019.
# 2. Demand for bike is high between may to oct for year 2018 and 2019.
# 3. From Boxchart we can also see that there is no outliers present.

In [ ]:
# Lets plot pairplots to have better to see if cnt is linearly  depend upon is independet variables. 
numerical_variabels=['cnt', 'temp', 'atemp', 'hum','windspeed']
sns.pairplot(data=bike_refined_df,vars=numerical_variabels, kind="reg")
plt.show()

Inferences:
1. We can see that we have linealy dependecy between temp, atemp and count.
2. we can also see that temp and atemp is highly correlated.

#  Barcharts for categorical variables to see demands

In [ ]:
plot_bar_graphs(bike_refined_df,'season')

Inferences:
1. Fall  seasons is having high number bike rental.
2. Bike rental in 2019 has increaded for every seasion compared to 2018.

In [ ]:
plot_bar_graphs(bike_refined_df,'mnth')

Inferences:
1. May to oct is having number of bike registration.
2. Bike registration in 2019 has increaded for every month compared to 2018.

In [ ]:
plot_bar_graphs(bike_refined_df,'weathersit')

Inferences:
1. People prefer bike rental when weather situations is good. 
2. Bike registration in 2019 has increaded  for every weather situtation 2018.

In [ ]:
plot_bar_graphs(bike_refined_df,'weekday')

Inferences:
1. People don't prefer bike rental on weekends. 

In [ ]:
plot_bar_graphs(bike_refined_df,'holiday')

Inferences:
1. People dont prefer renting bike holiday.

In [ ]:
plot_bar_graphs(bike_refined_df,'workingday')

Inferences:
1. People prefer bike rental on working days.

Lets draw heatMap to indentify is there any correlation.

In [ ]:
# Lets plot the corrlation matrix(heatmap)
plt.figure(figsize=(25,10))
sns.heatmap(bike_refined_df.corr(), cmap='BuGn', annot = True)
plt.show()

Inferences from heatmap: 
1. Since Count is sum of casual and registed. That we can infer from  heatmap as casual and registered are highly correleted with cnt. 
2. holiday, hum, and windspeed is negatively correlated. 
3. Indepedent variable temp and atemp is highly correlated(0.99). This show that there is high possiblity that one may have derived from another,One of the them can be dropped. Will use VIP and p values to drop this.

In [ ]:
bike_refined_df.shape

## Dummy Variables

We will create Dummy variables for these 4 categorical variables 'mnth', 'weekday', 'season' & 'weathersit'.

Creating dummy variables for categorical data is crucial in machine learning since most algorithms require numerical input. Categorical data in columns like 'mnth', 'weekday', 'season', and 'weathersit' need to be transformed into a numerical format to avoid ordinal assumptions that could mislead the model. Dummy variables, which create binary columns for each category, ensure that no false ordinal relationship is inferred.

We will have to convert them into 'category' data types, before creating dummy variables.

Before generating these dummy variables, converting the categorical columns to the 'category' data type offers significant benefits. This conversion reduces memory usage and speeds up computations compared to keeping the data in an 'object' (string) format or numerical types(assumed categorical types). Moreover, using the 'category' data type ensures that the data is consistently treated as categorical, maintaining data integrity throughout the preprocessing steps.

In [ ]:
bike_refined_df['season']=bike_refined_df['season'].astype('category')
bike_refined_df['weathersit']=bike_refined_df['weathersit'].astype('category')
bike_refined_df['mnth']=bike_refined_df['mnth'].astype('category')
bike_refined_df['weekday']=bike_refined_df['weekday'].astype('category')

In [ ]:
bike_refined_df['season'].replace({1:"spring", 2:"summer", 3:"fall", 4:"winter"},inplace = True)
bike_refined_df['weathersit'].replace({1:'good',2:'moderate',3:'bad',4:'severe'},inplace = True)

bike_refined_df['mnth'].replace({1: 'jan',2: 'feb',3: 'mar',4: 'apr',5: 'may',6: 'jun',
                  7: 'jul',8: 'aug',9: 'sept',10: 'oct',11: 'nov',12: 'dec'},inplace = True)

bike_refined_df['weekday'].replace({0: 'sun',1: 'mon',2: 'tue',3: 'wed',4: 'thu',5: 'fri',6: 'sat'},inplace = True)
bike_refined_df.head()

In [ ]:
bike_refined_df.info()


The code provided performs the following three tasks:

1. **Create Dummy Variables**: Converts categorical variables into dummy/indicator variables.
2. **Drop Original Variable**: Removes the original categorical variable after converting it to dummy variables.
3. **Drop First Dummy Variable**: Avoids the dummy variable trap by dropping the first dummy variable for each set of dummies created.

### 🌟 Explanation

Here's a step-by-step explanation with a focus on each task:

#### 1️⃣ Create Dummy Variables

**Creating dummy variables** means converting categorical variables into a series of binary (0 or 1) columns. Each unique category in the original variable becomes a new column.


#### 2️⃣ Drop Original Variable

After creating dummy variables, the original categorical variable is redundant and can be dropped. This is implicitly handled by `pd.get_dummies()` when creating dummies directly from the DataFrame.


#### 3️⃣ Drop First Dummy Variable

To avoid multicollinearity (dummy variable trap), we drop the first dummy variable of each set. This ensures that the dummy variables are not linearly dependent.

By following these steps, the original categorical variables are transformed into a format suitable for machine learning models, avoiding issues like multicollinearity.

In [ ]:
# This code does 3 things:
# 1) Create Dummy variable
# 2) Drop original variable for which the dummy was created
# 3) Drop first dummy variable for each set of dummies created.

bike_refined_df = pd.get_dummies(bike_refined_df, drop_first=True)
bike_refined_df.info()

## SPLITTING THE DATA

Splitting the data to Train and Test: - We will now split the data into TRAIN and TEST 

We will use train_test_split method from sklearn package for this using the ratio (70:30)

In [ ]:
# Check the shape before spliting

bike_refined_df.shape



In [ ]:
# importing train_test_split library from sklearn model selection package
from sklearn.model_selection import train_test_split

In [ ]:
# specified 'random_state' so that the train and test data set always have the same rows, respectively

np.random.seed(0)
df_train, df_test = train_test_split(bike_refined_df, train_size = 0.70, test_size = 0.30, random_state = 777)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train.shape

In [ ]:
df_test.shape

Perform the EDA on Training (df_train) Dataset

Visualising Numeric Variables using pairplot of all the numeric variables.

In [ ]:
bike_num=df_train[[ 'temp', 'atemp', 'hum', 'windspeed','cnt']]

sns.pairplot(bike_num, diag_kind='kde')
plt.show()

### Observation

The above PairPlot tells us that there is likely a Linear relationship between 'temp','atemp' and 'cnt'

## Visualising Catagorical Variables

Building boxplot of all categorical variables (before creating dummies) againt the target variable 'cnt' to see how each of the predictor variable stackup against the target variable.



In [ ]:
# Build boxplot of all categorical variables (before creating dummies) againt the target variable 'cnt' using bike_df
# to see how each of the predictor variable stackup against the target variable.

plt.figure(figsize=(25, 10))
plt.subplot(2,3,1)
sns.boxplot(x = 'season', y = 'cnt', data = bike_df)
plt.subplot(2,3,2)
sns.boxplot(x = 'mnth', y = 'cnt', data = bike_df)
plt.subplot(2,3,3)
sns.boxplot(x = 'weathersit', y = 'cnt', data = bike_df)
plt.subplot(2,3,4)
sns.boxplot(x = 'holiday', y = 'cnt', data = bike_df)
plt.subplot(2,3,5)
sns.boxplot(x = 'weekday', y = 'cnt', data = bike_df)
plt.subplot(2,3,6)
sns.boxplot(x = 'workingday', y = 'cnt', data = bike_df)
plt.show()

### Inference Summary 🚲

We analyzed 6 categorical variables in the dataset using a Box plot to study their effect on the dependent variable, `cnt`. Here are the key insights derived:

#### 📅 Season
- **Insight:** Season 3 accounts for approximately 32% of bike bookings with a median of over 5000 bookings over two years.
- **Comparison:** Season 2 and Season 4 follow with 27% and 25% of total bookings, respectively.
- **Conclusion:** Season is a significant predictor for bike bookings.

#### 📆 Month
- **Insight:** About 10% of bike bookings occur during months 5, 6, 7, 8, and 9, with a median of over 4000 bookings per month.
- **Conclusion:** Month shows a noticeable trend in bookings and can be a valuable predictor.

#### 🌤 Weather Situation
- **Insight:** Weather situation 1 (weathersit1) corresponds to nearly 67% of bike bookings with a median close to 5000 bookings over two years.
- **Comparison:** Weather situation 2 (weathersit2) follows with 30% of total bookings.
- **Conclusion:** Weather situation is a strong predictor of bike bookings.

#### 📅 Holiday
- **Insight:** Around 97.6% of bike bookings occur on non-holidays, indicating a clear bias.
- **Conclusion:** Holiday status is not a reliable predictor for bike bookings.

#### 📅 Weekday
- **Insight:** Bookings are relatively uniform across weekdays, ranging between 13.5% to 14.8% with medians between 4000 to 5000 bookings.
- **Conclusion:** The weekday variable may have a marginal influence, and the model should decide its inclusion.

#### 🏢 Working Day
- **Insight:** Approximately 69% of bike bookings occur on working days, with a median close to 5000 bookings over two years.
- **Conclusion:** Working day is a significant predictor for bike bookings.

### Comparison Table 🛠️

| Variable    | % of Total Bookings | Median Bookings | Predictive Strength    |
|-------------|---------------------|-----------------|------------------------|
| **Season**  | 32% (Season 3)      | >5000           | High                   |
| **Month**   | 10% (May-Sep)       | >4000           | Moderate               |
| **Weather** | 67% (weathersit1)   | ~5000           | High                   |
| **Holiday** | 97.6% (Non-holidays)| -               | Low                    |
| **Weekday** | 13.5%-14.8%         | 4000-5000       | Uncertain              |
| **Workday** | 69% (Workdays)      | ~5000           | High                   |

### Conclusion 📝

The analysis indicates that **Season**, **Month**, **Weather Situation**, and **Working Day** are strong predictors of bike bookings. **Holiday** is not a reliable predictor, and the influence of **Weekday** is uncertain, best left for the model to decide.


This approach ensures a clear understanding of each variable's impact and aids in the decision-making process for predictive modeling.

## Correlation Matrix

Let's use the correlation coefficients to see which variables are highly correlated. 

Note: here we are considering only those variables  that were chosen for analysis from bike_refined_df

In [ ]:
# Let's check the correlation coefficients to see which variables are highly correlated. Note:
# here we are considering only those variables  that were chosen for analysis from bike_refined_df

plt.figure(figsize = (25,20))
sns.heatmap(bike_refined_df.corr(), annot = True, cmap="RdBu")
plt.show()

### Observations

🔍 **Analyzing Collinearity by Understand a Heatmap:**
- The heatmap provides a clear visualization of multicollinear variables and highlights those with high collinearity with the target variable. 📊

🔄 **Using the Heatmap During Model Building:**
- We will frequently reference this heatmap while constructing our linear model. 🏗️
- By cross-checking correlated values with the heatmap, along with evaluating **Variance Inflation Factor (VIF)** and **p-values**, we can accurately determine which variables to include or exclude from our model. ✅❌

### Process Overview

Let's illustrate the process with a step-by-step approach:

1. **Visualizing Collinearity:**
    - Use a heatmap to display the correlation matrix of all variables.
    - Identify variables that are highly correlated with each other and with the target variable.

2. **Building the Linear Model:**
    - Refer to the heatmap to validate correlated values.
    - Calculate the VIF to assess the multicollinearity of each variable.
    - Evaluate p-values to check the significance of each variable.

3. **Variable Selection:**
    - Use the combined insights from the heatmap, VIF, and p-values.
    - Select or eliminate variables to refine the model.


### Key Points

| Aspect                 | Description                                            |
|------------------------|--------------------------------------------------------|
| **Heatmap**            | Visualizes multicollinearity between variables.        |
| **VIF**                | Measures the multicollinearity of each variable.       |
| **p-value**            | Determines the significance of each variable.          |
| **Variable Selection** | Combines heatmap, VIF, and p-values for optimal choice.|


Using this structured approach ensures a robust and interpretable linear model. 🌟

Bivariate analysis

Barcharts for categorical variables to see demands

## SCALING THE FEATURES



In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

scaler: This is an instance of a scaler object, likely from a library such as scikit-learn (sklearn.preprocessing.MinMaxScaler, for example). Scaling is a common preprocessing step in machine learning workflows, where you normalize the features to a similar scale to improve the performance of models, especially those sensitive to the scale of input features.

In [ ]:
# Checking the columns before scaling
df_train.columns


In [ ]:
# Checking the values before scaling
df_train.head()

### Apply scaler() to all the numeric variables

In [ ]:



# the names of the numeric variables/columns that you want to scale
num_vars = ['temp', 'atemp', 'hum', 'windspeed','cnt']

df_train[num_vars] = scaler.fit_transform(df_train[num_vars])


LINEAR MODEL BUILDING

Dividing into X and Y sets for the model building

In [ ]:
y_train = df_train.pop('cnt')
X_train = df_train

RFE
Recursive feature elimination: We will be using the LinearRegression function from SciKit Learn for its compatibility with RFE (which is a utility from sklearn)

In [ ]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
# Running RFE with the output number of the variable equal to 15
LR = LinearRegression()
estimator = LR.fit(X_train, y_train)



In [ ]:
rfe = RFE(estimator, n_features_to_select=15)   # running RFE
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

### listing recommended columns

In [ ]:
# listing recommended columns
col = X_train.columns[rfe.support_]
col

### listing ignored columns

In [ ]:
# listing ignored columns
X_train.columns[~rfe.support_]

In [ ]:
col2 = ['yr', 'holiday', 'workingday', 'temp', 'hum', 'windspeed','season_summer', 'season_winter', 'mnth_aug', 'mnth_sept', 'mnth_oct','weekday_sat', 'weekday_sun', 'weathersit_moderate', 'weathersit_bad']

In [ ]:
# Creating X_test dataframe with RFE selected variables
X_train_rfe = X_train[col2]


## using 'STATS MODEL' let us iteratively build models

Model 1

In [ ]:
# Check for the VIF values of the feature variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [ ]:
# util to calculate VIF given training df
def compute_VIF(train_df):
    # Create a dataframe that will contain the names of all the feature variables and their respective VIFs
    vif = pd.DataFrame()
    vif['Features'] = train_df.columns
    vif['VIF'] = [variance_inflation_factor(train_df.values, i) for i in range(train_df.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    vif

In [ ]:
compute_VIF(X_train[col])

In [ ]:
X_train_rfe.info()

In [ ]:
import statsmodels.api as sm



In [ ]:
def fit_OLS_model(y_train, X_train_rfe):

    # Add a constant
    X_train_lm = sm.add_constant(X_train_rfe)

    # Create a first fitted model
    lr = sm.OLS(y_train, X_train_lm).fit()
    
    return lr

In [ ]:
lr1 = fit_OLS_model(y_train,X_train_rfe)


In [ ]:
lr1.params()


In [ ]:
# Print a summary of the linear regression model obtained
print(lr1.summary())